In [4]:
import time
start_time = time.time()
import torch
end_time = time.time()

print((end_time - start_time)*1000)

0.014066696166992188


In [41]:
start_time = time.time()
z = torch.randn(10,6).to('cpu')
r = z[:,0]
p = z*z*z*z*z*z
end_time = time.time()
print((end_time - start_time)*1000)

0.5311965942382812


In [1]:
t = sum([1,3,4,5,6,7,8,9])
print(t)

43


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time

class LSTMAttn(nn.Module):
    def __init__(self, input_size=38, hidden_size=64, num_layers=2, dropout=0.1, output_size=1):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )
        self.dropout = nn.Dropout(dropout)
        self.attn_linear = nn.Linear(hidden_size, hidden_size)
        self.fc_out = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, (h, c) = self.lstm(x)   # [B, L, hidden]
        query = h[-1]                    # last layer’s hidden: [B, hidden]
        query_t = self.attn_linear(query) # [B, hidden]
        attn_scores = torch.bmm(lstm_out, query_t.unsqueeze(2)).squeeze(-1)  # [B, L]
        attn_weights = F.softmax(attn_scores, dim=1)                         # [B, L]
        context = lstm_out * attn_weights.unsqueeze(-1)                      # [B, L, hidden]
        context = context.sum(dim=1)                                         # [B, hidden]
        context = self.dropout(context)
        out = self.fc_out(context)  # -> [B, output_size]
        return out

# Model instantiation
model = LSTMAttn(input_size=72, hidden_size=144, num_layers=2, dropout=0.1, output_size=1)
x = torch.randn(256, 100, 72)  # Batch size=32, Sequence length=100, Feature size=38

# Measure inference time
model.eval()
with torch.no_grad():
    start_time = time.perf_counter()
    output = model(x)
    end_time = time.perf_counter()

print(f"Inference Time: {end_time - start_time:.6f} seconds")
#compute infernce time in ms 
print((end_time - start_time)*1000)
print(f"Output Shape: {output.shape}")


Inference Time: 0.025056 seconds
25.055630947463214
Output Shape: torch.Size([256, 1])


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time

class LSTMAttn(nn.Module):
    def __init__(self, input_size=38, hidden_size=64, num_layers=2, dropout=0.1, output_size=1):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )
        self.dropout = nn.Dropout(dropout)
        self.attn_linear = nn.Linear(hidden_size, hidden_size)
        self.fc_out = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, (h, c) = self.lstm(x)   # [B, L, hidden]
        query = h[-1]                    # last layer’s hidden: [B, hidden]
        query_t = self.attn_linear(query) # [B, hidden]
        attn_scores = torch.bmm(lstm_out, query_t.unsqueeze(2)).squeeze(-1)  # [B, L]
        attn_weights = F.softmax(attn_scores, dim=1)                         # [B, L]
        context = lstm_out * attn_weights.unsqueeze(-1)                      # [B, L, hidden]
        context = context.sum(dim=1)                                         # [B, hidden]
        context = self.dropout(context)
        out = self.fc_out(context)  # -> [B, output_size]
        return out

# Model instantiation
model = LSTMAttn(input_size=72, hidden_size=144, num_layers=2, dropout=0.1, output_size=1)
x = torch.randn(256, 100, 72)  # Batch size=256, Sequence length=100, Feature size=72
y = torch.randn(256, 1)  # True labels (target) for loss calculation

# Measure forward pass time
model.eval()
with torch.no_grad():
    start_time = time.perf_counter()
    output = model(x)
    end_time = time.perf_counter()

forward_time = (end_time - start_time) * 1000  # Convert to milliseconds
print(f"Inference (Forward Pass) Time: {forward_time:.6f} ms")

# Measure backward pass time (for training)
model.train()  # Set model to training mode
criterion = nn.MSELoss()  # Use Mean Squared Error loss (you can choose another loss)
optimizer = torch.optim.Adam(model.parameters())  # Optimizer

# Forward pass again
start_time = time.perf_counter()
output = model(x)
loss = criterion(output, y)
end_time = time.perf_counter()

forward_pass_time = (end_time - start_time) * 1000  # Time for forward pass in ms
print(f"Forward Pass (Training) Time: {forward_pass_time:.6f} ms")

# Backward pass
optimizer.zero_grad()  # Clear previous gradients
start_time = time.perf_counter()
loss.backward()  # Compute gradients
end_time = time.perf_counter()

backward_time = (end_time - start_time) * 1000  # Time for backward pass in ms
print(f"Backward Pass (Backpropagation) Time: {backward_time:.6f} ms")

# Parameter update (optimizer step)
start_time = time.perf_counter()
optimizer.step()  # Update parameters
end_time = time.perf_counter()

update_time = (end_time - start_time) * 1000  # Time for parameter update in ms
print(f"Parameter Update Time: {update_time:.6f} ms")


Inference (Forward Pass) Time: 26.329317 ms
Forward Pass (Training) Time: 40.561391 ms
Backward Pass (Backpropagation) Time: 72.084657 ms
Parameter Update Time: 2.951323 ms
